In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
# /kaggle/input/prokabadi/ProKabadi_match_level_details/DS_events.csv
players_df = pd.read_csv("/kaggle/input/prokabadi/ProKabadi_match_level_details/DS_players.csv")
teams_df = pd.read_csv("/kaggle/input/prokabadi/ProKabadi_match_level_details/DS_team.csv")
matches_df = pd.read_csv("/kaggle/input/prokabadi/ProKabadi_match_level_details/DS_match.csv")

In [ ]:
players_df.head()

In [ ]:
players_df.shape

In [ ]:
teams_df.head()

In [ ]:
teams_df.shape

In [ ]:
teams_df["name"].value_counts()

In [ ]:
# Pick releavent columns for teams
teams_df.info()
cols = ["id",
    "match_id",
	"name",
	"score",
	"stats.points.total",
	"stats.raids.successful",
	"stats.raids.total",
	"stats.raids.unsuccessful",
	"stats.tackles.successful",
	"stats.tackles.super_tackles",
	"stats.tackles.total",
	"stats.tackles.unsuccessful"]
teams_df = teams_df[cols]

In [ ]:
matches_df.head()

In [ ]:
matches_df.shape

In [ ]:
# Lets pick releavent columns
cols = ['match_id',
        'result',
		'player_id_of_the_match',
		'series_id',
        'series_name',
		'toss_winner',
		'home_team_id']
matches_df = matches_df[cols]

In [ ]:
matches_df.head()

In [ ]:
matches_df.isnull().sum()

In [ ]:
matches_df["winner"] = matches_df["result"].map(
    lambda x: x.split("beat")[0].rstrip() \
            if "beat" in str(x) else x)

matches_df["loser"] = matches_df["result"].map(
    lambda x: x.split("beat")[1].split("(")[0].strip() \
            if "beat" in str(x) else x)

matches_df["result"] = matches_df["result"].map(
    lambda x: "Match Tied" \
            if "Match Drawn" in str(x) else x)


In [ ]:
cond = matches_df["result"] != "Match Tied"
matches_df = matches_df[cond]

In [ ]:
winner_count = matches_df.groupby("winner")["winner"].count()
winner_count[:20].plot(kind="bar")

In [ ]:
matches_df.head()

In [ ]:
matches_df = pd.merge(left=matches_df, 
	right=teams_df, 
	how="left",
	left_on=["match_id", "winner"],
	right_on=["match_id", "name"],
	suffixes=('', '_winner')
)

matches_df = pd.merge(left=matches_df, 
	right=teams_df, 
	how="left",
	left_on=["match_id", "loser"],
	right_on=["match_id", "name"],
	suffixes=('', '_loser')
)


In [ ]:
matches_df.head()

#### Let's see if home team has a winning advantage

In [ ]:
home_df = pd.concat((matches_df["id"],
           matches_df["home_team_id"]), axis=1)
home_df.dropna(axis=0, inplace=True)
(home_df["id"] == home_df["home_team_id"]).sum() \
                        / home_df.shape[0]

##### Equal oddds, home team has no advantage over guest team

#### Lets see if winning toss affects game outcome

In [ ]:
home_df = pd.concat((matches_df["id"],
           matches_df["toss_winner"]), axis=1)
home_df.dropna(axis=0, inplace=True)
(home_df["id"] == home_df["toss_winner"]).sum() \
                        / home_df.shape[0]

##### Winning toss also has no influence on the outcome

#### Best performing teams for each season

In [ ]:
matches_df["winner"].groupby(
    matches_df["series_name"]).agg(pd.Series.mode)